In [6]:
print("First Video of Fine tuning")

First Video of Fine tuning


In [7]:
# Step1: Create & Setup hugging face API token in Collab

In [8]:
!pip install unsloth # install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git # Also get the latest version Unsloth!

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.0/130.0 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-x24juguu
  Running command git clone --fil

In [9]:
!pip uninstall -y unsloth unsloth-zoo
!pip install --no-cache-dir "unsloth[all]"

Found existing installation: unsloth 2025.8.10
Uninstalling unsloth-2025.8.10:
  Successfully uninstalled unsloth-2025.8.10
Found existing installation: unsloth_zoo 2025.8.9
Uninstalling unsloth_zoo-2025.8.9:
  Successfully uninstalled unsloth_zoo-2025.8.9
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 27.4 MB/s eta 0:00:00


In [10]:
# Step3: Import necessary libraries
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [11]:
# Step4: Check HF token
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
login(hf_token)

In [12]:
# Optional: Check GPU availability
# Test if CUDA is available
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
GPU device: Tesla T4


In [13]:
# Step5: Setup pretrained DeepSeek-R1

model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
max_sequence_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_sequence_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
)

==((====))==  Unsloth 2025.8.10: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [14]:
# Step6: Setup system prompt
prompt_style = """
Below is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.

Before crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.

### Task:
You are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.

### Query:
{}

### Answer:
<think>{}
"""

In [36]:
# Step7: Run Inference on the model

# Define a test question
question = """Scientists are developing a new non-steroidal anti-inflammatory drug for osteoarthritis, aiming for higher potency but the same efficacy as ibuprofen to reduce gastrointestinal side effects. If ibuprofen is represented by curve C in the figure and the desired therapeutic effect is marked by the dashed line Y, what curve would represent the new drug that fulfills these criteria?"""

FastLanguageModel.for_inference(model)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)


print(response)



["<｜begin▁of▁sentence｜>\nBelow is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.\n\nBefore crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.\n\n### Task:\nYou are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.\n\n### Query:\nScientists are developing a new non-steroidal anti-inflammatory drug for osteoarthritis, aiming for higher potency but the same efficacy as ibuprofen to reduce gastrointestinal side effects. If ibuprofen is represented by curve C in the figure and the desired therapeutic effect is marked by the dashed line Y, what curve would represent the new drug that fulfills these criteria?\n\n### Answer:\n<think>\nAlright, let's thi

In [37]:
print(response[0].split("### Answer:")[1])


<think>
Alright, let's think about this. We have ibuprofen, which is represented by curve C. The goal is to find a new drug that's more potent but doesn't cause more gastrointestinal issues. So, the new drug should be stronger, but it shouldn't be any worse for the stomach. 

First, let's figure out what potency means in this context. It's about how much of the drug is needed to achieve the same therapeutic effect as ibuprofen. So, if we need a higher potency, that means we can use less of the new drug to get the same effect. 

Now, let's think about efficacy. The dashed line Y represents the therapeutic effect we want. For ibuprofen, curve C shows that it hits Y at a certain dose. So, if the new drug is more potent, it should reach Y at a lower dose than ibuprofen.

But wait, we don't want the new drug to be worse in terms of side effects, especially gastrointestinal ones. So, we need to ensure that the new drug doesn't exceed the same limits as ibuprofen when it comes to side effect

In [18]:
# Step8: Setup fine-tuning

# Load Dataset
medical_dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split = "train[:500]", trust_remote_code = True)

In [20]:
medical_dataset[7]

{'Question': 'Scientists are developing a new non-steroidal anti-inflammatory drug for osteoarthritis, aiming for higher potency but the same efficacy as ibuprofen to reduce gastrointestinal side effects. If ibuprofen is represented by curve C in the figure and the desired therapeutic effect is marked by the dashed line Y, what curve would represent the new drug that fulfills these criteria?',
 'Complex_CoT': "Okay, so I need to figure out the characteristics of this new drug compared to ibuprofen. Let's see, we're looking at two main things here: potency and efficacy. If I remember correctly, potency is about how much of a drug we need to get a particular effect. So if a drug is more potent, we need less of it to get the job done. Efficacy, on the other hand, is about how good the drug is at its best; it's the maximum effect it can achieve.\n\nNow, for this new drug, they want it to be more potent than ibuprofen. That means it should work at a lower dose. But they still want it to be 

In [21]:
EOS_TOKEN = tokenizer.eos_token  # Define EOS_TOKEN which tells the model when to stop generating text during training
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [22]:
### Finetuning
# Updated training prompt style to add </think> tag
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""





In [23]:
# Prepare the data for fine-tuning

def preprocess_input_data(examples):
  inputs = examples["Question"]
  cots = examples["Complex_CoT"]
  outputs = examples["Response"]

  texts = []

  for input, cot, output in zip(inputs, cots, outputs):
    text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
    texts.append(text)

  return {
      "texts" : texts,
  }

In [24]:
finetune_dataset = medical_dataset.map(preprocess_input_data, batched = True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [35]:
finetune_dataset["texts"][7]

"Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\nPlease answer the following medical question.\n\n### Question:\nScientists are developing a new non-steroidal anti-inflammatory drug for osteoarthritis, aiming for higher potency but the same efficacy as ibuprofen to reduce gastrointestinal side effects. If ibuprofen is represented by curve C in the figure and the desired therapeutic effect is marked by the dashed line Y, what curve would represent the new drug that fulfills these criteria?\n\n### Response:\n<think>\nOkay, so I need to figure out the characteristics of this new drug compared to ibuprofen. Let's 

In [26]:
# Step9: Setup/Apply LoRA finetuning to the model

model_lora = FastLanguageModel.get_peft_model(
    model = model,
    r = 16,
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3047,
    use_rslora = False,
    loftq_config = None
)

Unsloth 2025.8.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [27]:
# Add this before creating the trainer
if hasattr(model, '_unwrapped_old_generate'):
    del model._unwrapped_old_generate

In [32]:
def formatting_function(examples):
    # examples is a dictionary where keys are column names and values are lists
    # We want to return a list of formatted strings, one for each example in the batch
    return [str(text) for text in examples["texts"]] # Explicitly cast to string

trainer = SFTTrainer(
    model = model_lora,
    tokenizer = tokenizer,
    train_dataset = finetune_dataset,
    dataset_text_field = "texts", # Keep this as it specifies the column containing the formatted text
    max_seq_length = max_sequence_length,
    dataset_num_proc = 1,
    formatting_func = formatting_function, # Use the defined function

    # Define training args
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 1,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir = "outputs",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/500 [00:00<?, ? examples/s]

In [33]:
# Setup WANDB
from google.colab import userdata
wnb_token = userdata.get("WANDB_API_TOKEN")
# Login to WnB
wandb.login(key=wnb_token) # import wandb
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-on-Medical-CoT-Dataset',
    job_type="training",
    anonymous="allow"
)


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bilaltaf (bilaltaf-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [34]:
# Start the fine-tuning process
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.932200
20,1.424800
30,1.406800
40,1.364700
50,1.391200
60,1.370500


In [38]:
wandb.finish()

train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▂▁▁▂▁
train/learning_rate,█▇▅▄▂▁
train/loss,█▂▂▁▁▁
total_flos,1.6686758448119808e+16
train/epoch,0.96
train/global_step,60
train/grad_norm,0.2562
train/learning_rate,0.0
train/loss,1.3705


In [39]:
# Step10: Testing after fine-tuning
question = """Scientists are developing a new non-steroidal anti-inflammatory drug for osteoarthritis, aiming for higher potency but the same efficacy as ibuprofen to reduce gastrointestinal side effects. If ibuprofen is represented by curve C in the figure and the desired therapeutic effect is marked by the dashed line Y, what curve would represent the new drug that fulfills these criteria?"""

FastLanguageModel.for_inference(model_lora)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model_lora.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)

print(response)

["<｜begin▁of▁sentence｜>\nBelow is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.\n\nBefore crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.\n\n### Task:\nYou are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.\n\n### Query:\nScientists are developing a new non-steroidal anti-inflammatory drug for osteoarthritis, aiming for higher potency but the same efficacy as ibuprofen to reduce gastrointestinal side effects. If ibuprofen is represented by curve C in the figure and the desired therapeutic effect is marked by the dashed line Y, what curve would represent the new drug that fulfills these criteria?\n\n### Answer:\n<think>\nOkay, so there's t

In [40]:
print(response[0].split("### Answer:")[1])


<think>
Okay, so there's this new drug they're working on for osteoarthritis. It's supposed to be a non-steroidal anti-inflammatory drug, and they want it to be more potent than ibuprofen but not have the same level of gastrointestinal side effects. Interesting, right? So, they want it to be as effective as ibuprofen but with less of the stuff that makes people's stomachs upset.

Alright, let's think about what we know about ibuprofen. It's a classic NSAID, right? And it's represented by curve C in the figure. Now, what does that curve tell us? Well, in a typical dose-response curve, the y-axis usually represents the effect, and the x-axis represents the dose. So, if we're saying ibuprofen is represented by curve C, that means we're looking at how much of it is needed to get a certain effect level.

Now, let's get back to the new drug. They want it to be more potent, meaning for the same effect, it needs a lower dose. But they also want the same efficacy as ibuprofen in terms of the t

In [41]:
question = """A 46-year-old woman presents with an itchy rash on her legs and examination reveals white lace-like lines on the buccal mucosa. Based on these findings, what would a biopsy of the skin lesion most likely show?"""

FastLanguageModel.for_inference(model_lora)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model_lora.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)

print(response[0].split("### Answer:")[1])




<think>
Alright, so we have a 46-year-old woman who's got this itchy rash on her legs. That's interesting. And when we look at her mouth, we see these white lines on the buccal mucosa. Hmm, that's kind of unusual. Let's think about what could be going on here.

First off, the rash on her legs suggests some kind of skin issue. It's pretty common for people to have skin problems that aren't too serious, like contact dermatitis or something like that. But then there's this white lace-like stuff on her mouth. That's a bit more of a puzzle. I know that sometimes skin conditions can show up in different parts of the body, so maybe it's something that's linked.

Now, what comes to mind when I see white lace-like lines on the buccal mucosa? It sounds like it could be something called vitiligo. Vitiligo is a condition where the skin loses its pigment, creating these white patches. And it can affect various parts of the body, including the mouth. That's a possibility.

But wait, let's not jump 

In [ ]:
!pip install unsloth_zoo